## Hämta data från Bucket (Minio)

### Ladda paket: xlrd behövs för Excel

In [1]:
from minio import Minio
from minio.commonconfig import Tags
from io import BytesIO
import os
import pandas as pd

import tempfile

try:
    import xlrd
except:
    !pip install xlrd
    import xlrd

try:
    import openpyxl
except:
    !pip install openpyxl
    import openpyxl

try:
    import pyreadstat
except:
    !pip install pyreadstat
    import pyreadstat


In [2]:
!pip install sas7bdat
!pip install xport

## Funktioner 

För att läsa in data från bucket (minio) resp. att skriva till bucket.
Indata kan vara av typ "csv", "excel", eller "sas".

In [3]:
# function to write object and tags

def read_from_s3(bucket, filepath, datatype='csv'):
    """
    Read a file from an S3 bucket.

    :param bucket: The S3 bucket name.
    :param filepath: The path to the file within the bucket.
    :datatype: 'csv','excel',or 'sas'
    :return: Pandas dataframe  or None if the file does not exist.
    """
    # Create Minio client
    client = Minio(
        os.getenv("JUPYTERLAB_S3_ENDPOINT").replace("http://", ""),
        secure=False,
        access_key=os.getenv("JUPYTERLAB_S3_ACCESS_KEY_ID"),
        secret_key=os.getenv("JUPYTERLAB_S3_SECRET_ACCESS_KEY"),
    )

    # Check if the bucket exists
    found = client.bucket_exists(bucket)
    if not found:
        print("Bucket does not exist: %s" % (bucket))
        return None  # Failure

    # Check if the object exists
    try:
        obj = client.get_object(bucket, filepath)
        # Read the object into a bytes buffer
        buffer = BytesIO(obj.read())
        # Read the bytes buffer into a pandas dataframe
        if datatype=='csv':
            df = pd.read_csv(buffer)
            return df
        elif datatype == 'excel':
            df = pd.read_excel(buffer)
            return df
        elif datatype == 'sas':
            # create a temporary file
            with tempfile.NamedTemporaryFile(mode="wb", delete=False) as tmp:
                # write the bytes data to the file
                tmp.write(buffer.getvalue())
                # get the file name
                tmp_name = tmp.name
            df, metadata = pyreadstat.read_sas7bdat(tmp_name)
            return df
        else:
            print("Only csv, excel, or sas are accepted datatype. Please check your datatype.")
            return None
    except Exception as e:
        print(f"Error reading file {filepath} from bucket {bucket}: {str(e)}")
        return None  # Failure

def write_to_s3(bucket, filepath, tags={}, meta={}):
    # create Minio client and init bucket if necessary
    client = Minio(
        os.getenv("JUPYTERLAB_S3_ENDPOINT").replace("http://", ""),
        secure=False,
        access_key=os.getenv("JUPYTERLAB_S3_ACCESS_KEY_ID"),
        secret_key=os.getenv("JUPYTERLAB_S3_SECRET_ACCESS_KEY"),
    )
    found = client.bucket_exists(bucket)
    if not found:
        print("Bucket does not exist: %s" % (bucket))
        return False  # failure

    # init tags
    t = Tags(for_object=True)
    for k, v in tags.items():
        t[k] = v

    with open(filepath, mode="rb") as source_file:
        contents = source_file.read()

    client.put_object(
        bucket,
        filepath,
        data=BytesIO(contents),
        length=len(contents),
        tags=t,
        metadata=meta,
        content_type='application/csv'
    )
    return True  # success

### Ett exempel med SAS data

In [4]:
bucket_name = "XXXX"
file_path_in_s3 = "index_orig.sas7bdat"

In [5]:
df = read_from_s3(bucket_name, file_path_in_s3, datatype="sas")
df.tail()

,factKlonKey,Period,VariableId,VariableName,MeasureValue,BranchGroupId,BranchName,BranchCode,OutClass,IsFinal,MarknadCode,MarknadName
3583,1.088541e+09,2021M09,3.0,Prisindex,88.0,4132.0,Tillverkning av elektroniska komponenter och k...,26.1,26.1|5,0.0,5,Total (=Hemma+Export)
3584,1.138311e+09,2021M12,3.0,Prisindex,140.5,3949.0,Investeringsmetallvaruindustrin utom maskiner,25.1-4,25.1-4|5,0.0,5,Total (=Hemma+Export)
3585,1.219053e+09,2022M05,3.0,Prisindex,158.1,3949.0,Investeringsmetallvaruindustrin utom maskiner,25.1-4,25.1-4|5,0.0,5,Total (=Hemma+Export)
3586,1.219053e+09,2022M05,3.0,Prisindex,129.9,1977.0,Tillverkning av hushållsmaskiner och hushållsa...,27.5,27.5|5,0.0,5,Total (=Hemma+Export)
3587,1.219053e+09,2022M05,3.0,Prisindex,151.7,3467.0,"Tillverkning av fartyg, båtar, rälsfordon, luf...",30.1-4,30.1-4|5,0.0,5,Total (=Hemma+Export)


### Exempel med Excel fil

In [6]:
file2 = "PrisIndex.xlsx"

In [7]:
df2 = read_from_s3(bucket_name, file2, datatype="excel")
df2.tail()

,factKlonKey,Period,VariableId,VariableName,MeasureValue,BranchGroupId,BranchName,BranchCode,OutClass,IsFinal,MarknadCode,MarknadName
3583,1088541491,2021M09,3,Prisindex,88.0,4132,Tillverkning av elektroniska komponenter och k...,26.1,26.1|5,0,5,Total (=Hemma+Export)
3584,1138310933,2021M12,3,Prisindex,140.5,3949,Investeringsmetallvaruindustrin utom maskiner,25.1-4,25.1-4|5,0,5,Total (=Hemma+Export)
3585,1219053228,2022M05,3,Prisindex,158.1,3949,Investeringsmetallvaruindustrin utom maskiner,25.1-4,25.1-4|5,0,5,Total (=Hemma+Export)
3586,1219053212,2022M05,3,Prisindex,129.9,1977,Tillverkning av hushållsmaskiner och hushållsa...,27.5,27.5|5,0,5,Total (=Hemma+Export)
3587,1219053223,2022M05,3,Prisindex,151.7,3467,"Tillverkning av fartyg, båtar, rälsfordon, luf...",30.1-4,30.1-4|5,0,5,Total (=Hemma+Export)
